In [ ]:
from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

## Realizando a conexão com o MongoDB

Nesse momento vamos utilizar o código fornecido no MongoDB Atlas para realizarmos a conexão com o nosso banco de dados utilizando a biblioteca pymongo.

In [5]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = os.getenv("MONGODB_URI")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Criando o banco de dados e uma coleção

Conectados ao banco de dados, está tudo pronto para realizar operações nele. Primeiramente, vamos criar nossa base de dados chamada "db_produtos" e uma coleção dentro dela chamada "produtos". Vale lembrar que para um banco de dados do MongoDB ser efetivamente criado, ele precisa conter dados.

In [6]:
db = client["db_produtos"]
collection = db["produtos"]

In [7]:
client.list_database_names()

['db_produtos_desafio', 'admin', 'local']

In [8]:
product = {"produto": "computador", "quantidade": 77}
collection.insert_one(product)

In [9]:
collection.find_one()

{'_id': ObjectId('64d3d17ffb7781b7e7d5087b'),
 'produto': 'computador',
 'quantidade': 77}

In [10]:
client.list_database_names()

['db_produtos', 'db_produtos_desafio', 'admin', 'local']

## Extraindo os dados da API

Agora que nossa base de dados e coleção estão criadas, podemos prosseguir para a etapa de extração dos dados que desejamos adicionar a esse banco de dados.

In [11]:
import requests

response = requests.get("https://labdados.com/produtos")
response.json()[:2]

[{'Produto': 'Modelagem preditiva',
  'Categoria do Produto': 'livros',
  'Preço': 92.45,
  'Frete': 5.6096965236,
  'Data da Compra': '01/01/2020',
  'Vendedor': 'Thiago Silva',
  'Local da compra': 'BA',
  'Avaliação da compra': 1,
  'Tipo de pagamento': 'cartao_credito',
  'Quantidade de parcelas': 3,
  'lat': -13.29,
  'lon': -41.71},
 {'Produto': 'Iniciando em programação',
  'Categoria do Produto': 'livros',
  'Preço': 43.84,
  'Frete': 0.0,
  'Data da Compra': '01/01/2020',
  'Vendedor': 'Mariana Ferreira',
  'Local da compra': 'SP',
  'Avaliação da compra': 5,
  'Tipo de pagamento': 'cartao_credito',
  'Quantidade de parcelas': 1,
  'lat': -22.19,
  'lon': -48.79}]

In [12]:
len(response.json())

9435

## Adicionando os dados extraídos na coleção

Com os dados extraídos no formato JSON, que é compatível com o MongoDB, podemos adicioná-los facilmente ao nosso banco de dados. Podemos utilizar o método `insert_many()` para adicionar todos os dados extraídos de uma vez.

In [13]:
docs = collection.insert_many(response.json())

In [14]:
len(docs.inserted_ids)

9435

In [15]:
collection.count_documents({})

9436

**Removendo o registro de exemplo**

In [16]:
id_remover = collection.find_one()["_id"]

In [17]:
collection.delete_one({"_id": id_remover})

In [18]:
collection.find_one()

{'_id': ObjectId('64d3d182fb7781b7e7d5087c'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'lat': -13.29,
 'lon': -41.71}

In [19]:
client.close()